In [1]:
from datetime import datetime
import pandas as pd
from pathlib import Path
import pickle

In [2]:
data_dir = Path('../data')

In [3]:
csvs = [x for x in data_dir.iterdir() if 'timeseries' in str(x)]
counts = []
regions = []
results = pd.DataFrame(columns=['roi', 'new_deaths', 'new_cases', 'new_recover', 'abs_days', 'rel_days'])
i = 1
for csv in csvs:
    region = str(csv).split('.')[0].split('_')[-1]
    regions.append(region)
    df = pd.read_csv(csv)
    df = df.iloc[:-1]
    df = df[df['cum_cases']>=10]
    df['abs_days'] = df['dates2'].apply(lambda x: (datetime.strptime(x, '%m/%d/%y') - datetime.strptime('01/22/20', '%m/%d/%y')).days)
    df['rel_days'] = df['dates2'].apply(lambda x: (datetime.strptime(x, '%m/%d/%y') - datetime.strptime(df['dates2'].values[0], '%m/%d/%y')).days)
    counts.append(df.shape[0])
    subset = ['new_deaths', 'new_cases', 'new_recover']
    df[subset] = df[subset].clip(lower = 0)
    df['roi'] = i
    subset = ['roi', 'new_deaths', 'new_cases', 'new_recover', 'abs_days', 'rel_days']
    results = results.append(df[subset])
    i += 1

In [4]:
import pickle
with open(data_dir / 'stacked.pkl', 'wb') as f:
    pickle.dump({'data': results, 'regions': regions, 'counts': counts}, f)

In [5]:
results

,roi,new_deaths,new_cases,new_recover,abs_days,rel_days
48,1,0,10,0,48,0
49,1,0,3,1,49,1
50,1,0,1,0,50,2
51,1,0,8,0,51,3
52,1,0,3,0,52,4
...,...,...,...,...,...,...
80,171,0,2,0,80,8
81,171,0,0,0,81,9
82,171,0,2,0,82,10
83,171,0,0,3,83,11
